# Predict California house prices

### Install DS packages
- Un-comment the line below and run
- Then comment back and restart the notebook's kernel ▶️

In [ ]:
# !pip install catboost numpy==1.25 pandas scikit-learn matplotlib shap

In [ ]:
import catboost as cb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import data
California house prices and house features in 1990.
👉 More info: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

In [ ]:
from sklearn.datasets import fetch_california_housing
california = fetch_california_housing()
housing = pd.DataFrame(california.data, columns=california.feature_names)

# EDA
Explore the housing data

In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.isnull().sum()

In [ ]:
housing.hist()
plt.tight_layout()

In [ ]:
housing['AveRooms'].hist(range=[0,10])
print(f"Average number of rooms {housing['AveRooms'].mean():.2}")

In [ ]:
plt.scatter(x=housing['Longitude'], y=housing['Latitude']);
print("Can see the Bay Area - good sanity check!")

In [ ]:
housing['Latitude'].hist();
print("We have two regions, North (>36) and South (<36)")

# Feature Engineering

Now, let's assume (given our real estate market knowledge) we want to remove `Latitude` and `Longitude` features, and replace with new `Region` (`South` or `North`) featutre.

In [ ]:
def create_new_feature_region(latitude_north: float = 36.0) -> pd.DataFrame:
    # From a mask (True, False)
    is_north_cal = housing['Latitude'] > latitude_north
    # Create a new dataset with removed features
    updated_housing=housing.drop(columns=['Latitude', 'Longitude'])
    # Add a new feature based on masking
    updated_housing['Region_bool'] = is_north_cal 
    # Replace (True->North, False->South) -- useful if we will add more regions later. 
    updated_housing['Region']=updated_housing['Region_bool'].mask(updated_housing['Region_bool']==True,  'North').mask(updated_housing['Region_bool']==False, 'South')
    # Remove our masking column from the final dataset
    updated_housing.drop(columns=['Region_bool'], inplace=True)
    return updated_housing

In [ ]:
updated_housing = create_new_feature_region()
# print out some values to confirm
updated_housing.iloc[1209:12099]

! Note, we haven't looked at the target (house price in $100K yet) to avoid any bias.

In [ ]:
features, target = updated_housing, california.target

Split data into *test* 20%, *val* 20% (0.8*0.25), *train* 60% and load into Pool https://catboost.ai/en/docs/concepts/python-reference_pool

In [ ]:
from sklearn.model_selection import train_test_split
def return_train_val_test_pools() -> cb.Pool:
    features_train_val, features_test, target_train_val, target_test = train_test_split(features, target, test_size=0.2, random_state=1)
    features_train, features_val, target_train, target_val = train_test_split(features_train_val, target_train_val, test_size=0.25, random_state=1)
    train_dataset = cb.Pool(features_train, target_train, cat_features=['Region']) 
    val_dataset = cb.Pool(features_val, target_val, cat_features=['Region']) 
    test_dataset = cb.Pool(features_test, target_test, cat_features=['Region']) 
    print(f"Train size: {train_dataset.shape[0]/len(target)}, val size: {val_dataset.shape[0]/len(target)}, test size: {test_dataset.shape[0]/len(target)}")
    return train_dataset, val_dataset, test_dataset, features_test, target_test

In [ ]:
train_pool, val_pool, test_pool, features_test, target_test = return_train_val_test_pools()

In [ ]:
from statistics import mean 
plt.hist(train_pool.get_label())
print(f"Average house price in 1990 in the train dataset is {int(mean(train_pool.get_label()*100_000))} $")
print(f"According to Demographia, the average house price in California in 1990 was $194300.")

# Training
Load CatBoost with defaul parameters, more info https://catboost.ai/en/docs/concepts/python-reference_catboostregressor 

In [ ]:
model = cb.CatBoostRegressor()

Train with early stopping

In [ ]:
model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=20, verbose=50, plot=True)

# Testing

Let's test model prediction for a single house from the test dataset

In [ ]:
test_house = features_test.iloc[25] 
true_price = target_test[25] 
print(f"Test house:\n{test_house}")

In [ ]:
print(f"True price: ${int(true_price*100_000)}")
prediction = model.predict(test_house)
print(f"Model prediction: ${int(prediction*100_000)}")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
pred = model.predict(test_pool)
rmse = (np.sqrt(mean_squared_error(target_test, pred)))
r2 = r2_score(target_test, pred)
print("Testing performance")
print(f"RMSE: ${int(rmse*100_000)} -- our model root mean squared error, our average expected error on house prediction")
print(f"R2: {r2:.2f} -- a fraction of house prices that were well predicted by the model")

# Feature Importance

Let's look at which housing featues had the most impact on model predictions. Again, we will use our real estate market knowledge to see if the result is making sense.
We will use SHAP library for this: https://pypi.org/project/shap/ 

### What does this plot mean? 
- Positive SHAP value = driving the house prediction price up
- Red = large value of a features

Example: Large number of bedrooms (red) means higher predicted price.

In [ ]:
import shap
from tqdm.notebook import tqdm
explainer = shap.TreeExplainer(model)
shap_values = explainer(features_test)
shap.plots.beeswarm(shap_values)

We can even ask for explanations for a single house! 

In [ ]:
shap.initjs()
shap.plots.force(shap_values[25, ...])